In [ ]:
import sys
import os
import tensorflow.keras
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import preprocess_input
import pandas as pd
from sklearn.model_selection import train_test_split
import PIL.Image as Image
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True)
np.set_printoptions(threshold=sys.maxsize)

from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
import pandas as pd

df=pd.read_csv('/content/drive/My Drive/BachelorProject/dataframe/reducedData1.csv')

In [ ]:
temp1 = df.copy()
temp2 = df.copy()
temp3 = df.copy()
temp4 = df.copy()
temp5 = df.copy()

In [ ]:
temp1.drop(temp1[temp1.Label != 'Kitchen'].index, inplace=True)
temp2.drop(temp2[temp2.Label != 'Bedroom'].index, inplace=True)
temp3.drop(temp3[temp3.Label != 'Sport fields'].index, inplace=True)
temp4.drop(temp4[temp4.Label != 'Living room'].index, inplace=True)
temp5.drop(temp5[temp5.Label != 'Restaurant,Bar'].index, inplace=True)

In [ ]:
five_labels_df = pd.concat([temp1, temp2, temp3, temp4, temp5], ignore_index=True)

In [ ]:
five_labels_df.Label.value_counts()

Kitchen           599
Bedroom           543
Sport fields      538
Living room       524
Restaurant,Bar    521
Name: Label, dtype: int64

In [ ]:
five_labels_df = pd.read_csv('/content/drive/My Drive/BachelorProject/dataframe/five_labels_df.csv', index_col=0)

In [ ]:
five_labels_df.Label.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split

temp, test_set = train_test_split(five_labels_df, test_size=0.2, stratify=five_labels_df.Label)

train_set, validation_set = train_test_split(temp, test_size=0.2, stratify=temp.Label)

In [ ]:
from keras.layers import Dense, Activation, Flatten
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

def createModel():
  counter = 0
  resnet50 = tf.keras.applications.resnet50
  conv_model = resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
  for layer in conv_model.layers:
      counter += 1
      layer.trainable = False

  x = tf.keras.layers.Flatten()(conv_model.output)

  predictions = tf.keras.layers.Dense(5, activation='softmax')(x)
  full_model = tf.keras.models.Model(inputs=conv_model.input, outputs=predictions)

  return full_model

In [ ]:
from keras_preprocessing.image import ImageDataGenerator
import tensorflow as tf


datagen=ImageDataGenerator(rescale=1./255, preprocessing_function=tf.keras.applications.resnet50.preprocess_input)
train_generator=datagen.flow_from_dataframe(dataframe=train_set, x_col="FilePath", y_col="Label", class_mode="categorical", seed=42, 
                                            target_size=(224,224), color_mode='rgb', save_format='jpg', batch_size=10, shuffle=True)

validation_genarator=datagen.flow_from_dataframe(dataframe=validation_set, x_col="FilePath", y_col="Label", class_mode="categorical", seed=42, 
                                            target_size=(224,224), color_mode='rgb', save_format='jpg', batch_size=10, shuffle=True)

test_genarator=datagen.flow_from_dataframe(dataframe=test_set, x_col="FilePath", y_col="Label", class_mode="categorical", seed=42, 
                                            target_size=(224,224), color_mode='rgb', save_format='jpg', batch_size=10, shuffle=False)

Found 1744 validated image filenames belonging to 5 classes.
Found 436 validated image filenames belonging to 5 classes.
Found 545 validated image filenames belonging to 5 classes.


In [ ]:
full_model = createModel()
full_model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
                  metrics=['accuracy'])

94773248/94765736 [==============================] - 0s 0us/step


In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALIDATION=validation_genarator.n//validation_genarator.batch_size

In [ ]:
full_model.fit(train_generator,
          epochs=1,
          steps_per_epoch=STEP_SIZE_TRAIN,
          validation_data=validation_genarator,
          validation_steps=STEP_SIZE_VALIDATION)

full_model.save('/content/drive/My Drive/BachelorProject/saved_models/full_modellll')

174/174 [==============================] - 1042s 6s/step - loss: 2.1925 - accuracy: 0.3466 - val_loss: 1.3427 - val_accuracy: 0.4116


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: /content/drive/My Drive/BachelorProject/saved_models/full_modellll/assets


In [ ]:
from tensorflow import keras

model = keras.models.load_model('/content/drive/My Drive/BachelorProject/saved_models/full_modellll')

In [ ]:
model.summary

In [ ]:
def plotImages(image_arr):
  fig, axes = plt.subplots(1, 10, figsize=(20, 20))
  axes = axes.flatten()
  for img, ax in zip(image_arr, axes):
    ax.imshow(img)
    ax.axis('off')
  plt.tight_layout()
  plt.show()

In [ ]:
img, labels = next(train_generator)
plotImages(img)
print(labels)

In [ ]:
thisdict = {
  "Bedroom": 0,
  "Kitchen": 1,
  "Living room": 2,
  "Restaurant,Bar" : 3,
  "Sport fields" : 4
}

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

y_pred = model.predict(test_genarator)

predicted_labels = np.argmax(y_pred, axis=-1)

real_labels = [thisdict[elem] for elem in test_set.Label]

# accuracy_score(real_labels, predicted_labels)

In [ ]:
accuracy_score(real_labels, predicted_labels)

0.44220183486238535